In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from deepdrr.utils import test_utils
from deepdrr.two.renderer import *
from deepdrr.two.profiles import *

from deepdrr.two.renderable import *
from deepdrr.two.devices import *



In [3]:
f = test_utils.download_sampledata("CT-chest")
mf = Path("tests/resources/suzanne.stl")
mf2 = Path("tests/resources/threads.stl")
print(f)

Using downloaded and verified file: /home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd
/home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd


### DeepDRR v2 API proposal

#### v2 api changes

- Split frontend (with volume, mesh, and transform utilities) and backend (rendering engine).
- Intermediate immutable data structure for sending render commands from frontend to backend. Can be JSON serialized for parallel worker rendering.


#### Advantages of v2 api

- Separate the frontend (full of task specific utilities unrelated to deepdrr) from the actual rendering code with an interface.
- Ready for large scale dataset generation. Sample a large dataset quickly on a single machine using whatever utilities and libraries, then have simple cloud or cluster workers that render the images from the JSON snapshots.
- Fix confusion with ownership of Renderables that could only be present in one Projector at a time. Instead, make a single scene, have multiple render profiles targetting different subsets of renderables.
- Easier to make dedicated rendering servers like we made for PelvisVR, using standard JSON serialization protocol. (Opens the door for non-python frontends and backends, e.g. Unity frontend, C++ backend).
- Separate worker processes can run different render backends that work best on different compute (e.g. GPU for DRR raymarching, CPU OpenGL Mesa for rendering masks)
- Will be easier to eventually write efficient GPU rendering backends with pipelining for max throughput when running as a worker process, all render frames known upfront.
- Frontend "scenes" can have nice transform management utilities (e.g. transform tree/graph manager, like fledge) without complicating the backend (as opposed to the current flat list of Renderables in a Projector).
- Frontend could eventually be split out into separate library with utilities for volume and mesh manipulation that are not specific to rendering (e.g. mesh decimation, volume cropping, segmentation, etc).

#### Current status

- Frontend core data structures are mostly implemented
- DeferredRenderer JSON serialization is implemented
- Simple transform manager is implemented (TransformTree), eventually should be replaced with dedicated library like fledge

#### Future work
- Rendering backends are not implemented
- Frontend mesh/volume utilities are not implemented yet, but we would aim for feature parity with current deepdrr volume/mesh utilities.

#### See

- Examples below
- Core data structure implementation in [../deepdrr/two](../deepdrr/two)
- JSON serialization example in [./drr.json](./drr.json)


In [ ]:
# Example 1: The v2 api can be used similar to the v1 api-- no JSON serialization, render synchronously on the current thread

volume = Volume.from_nrrd(f)

mesh1 = Mesh.from_stl(mf, tags=["mesh1"])
mesh2 = Mesh.from_stl(mf2, tags=["mesh2"])

carm = MobileCArm()

drr_settings = DRRRenderSettings(
    width=512,
    height=512,
    neglog=True,
)
drr_prof = DRRRenderProfile(
    settings=drr_settings,
    renderer=SynchronousRenderer(),
    scene=GraphScene.from_flat_list([carm, volume, mesh1, mesh2]),
)

with drr_prof:
    mesh1.world_from_anatomical = geo.FrameTransform.from_translation([-30, 50, 200])
    
    for frame_idx in range(10):
        mesh2.world_from_anatomical = geo.FrameTransform.from_translation([0, 0, 10*frame_idx])
    
        carm.move_by(
            delta_isocenter=[0, 0, 10],
            delta_alpha=5*frame_idx,
            delta_beta=0,
            delta_gamma=0,
            degrees=False,
        )
    
        drr = drr_prof.render_drr()
        segs = drr_prof.render_seg(tags=["mesh1", "mesh2"])
    
    
        


In [4]:
# Example 2: Deferred rendering. Serialize render commands to a JSON file.
# Later, these JSON files would be loaded by a worker and rendered (not shown)

volume = Volume.from_nrrd(f)

mesh1 = Mesh.from_stl(mf, tags=["mesh1"])
mesh2 = Mesh.from_stl(mf2, tags=["mesh2"])

carm = MobileCArm()

# Create a scene with an arbitrary transform tree
graph = TransformTree()

graph.add(carm)
graph.add(volume)

volume.add(mesh1) # Parent the mesh to the volume
mesh1.add(mesh2) # Parent a mesh to another mesh

scene = GraphScene(graph) # Create a scene from the graph

drr_json_path = Path("drr.json")
rast_json_path = Path("rasterize.json")

# Create multiple renderers for the same scene, no confusion about which "owns" the objects

drr_settings = DRRRenderSettings(
    width=512,
    height=512,
    neglog=True,
)
drr_prof = DRRRenderProfile(
    settings=drr_settings,
    renderer=DeferredRenderer(drr_json_path), # Don't actually render, serialize to a JSON file for later cloud rendering
    scene=scene
)

# Serialize to a separate JSON for rendering masks so that it can be done in parallel on the CPU
rast_settings = RasterizeRenderSettings(
    width=512,
    height=512,
    max_mesh_hits=32,
)
rasterize_prof = RasterizeRenderProfile(
    settings=rast_settings,
    renderer=DeferredRenderer(rast_json_path), # Don't actually render, serialize to a JSON file for later cloud rendering
    scene=scene
)

with drr_prof, rasterize_prof:
    
    mesh1.world_from_anatomical = geo.FrameTransform.from_translation([-30, 50, 200])
    
    
    for frame_idx in range(10):
        mesh2.world_from_anatomical = geo.FrameTransform.from_translation([0, 0, 10*frame_idx])
    
        carm.move_by(
            delta_isocenter=[0, 0, 10],
            delta_alpha=5*frame_idx,
            delta_beta=0,
            delta_gamma=0,
            degrees=False,
        )
    
        drr_prof.render_drr(f"./drr_out/frame_{frame_idx:03d}.tiff")
    
        rasterize_prof.render_seg(f"./seg_out/frame_{frame_idx:03d}.json", tags=["mesh1", "mesh2"])
    
    
        
